#Scraping et import des données de mix énergétique de l'AIE
Module d'import des données des mixes énergétiques de production de l'énergie électrique mis à disposition par l'Agence Internationale de l'Energie.
https://www.iea.org/countries

##Initialisation

###Installations

In [9]:
!apt update
!apt install chromium-chromedriver
!pip install selenium
!pip install psycopg2

Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Get:3 https://dl.google.com/linux/chrome/deb stable InRelease [1,825 B]
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Hit:5 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:8 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:10 https://dl.google.com/linux/chrome/deb stable/main amd64 Packages [1,083 B]
Hit:11 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:12 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:13 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [1,242 kB]
Get:

In [10]:
!wget https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb && apt install ./google-chrome-stable_current_amd64.deb

--2023-11-23 14:16:01--  https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
Resolving dl.google.com (dl.google.com)... 173.194.216.91, 173.194.216.190, 173.194.216.93, ...
Connecting to dl.google.com (dl.google.com)|173.194.216.91|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 103884100 (99M) [application/x-debian-package]
Saving to: ‘google-chrome-stable_current_amd64.deb.1’

google-chrome-stabl 100%[===================>]  99.07M   128MB/s    in 0.8s    

2023-11-23 14:16:01 (128 MB/s) - ‘google-chrome-stable_current_amd64.deb.1’ saved [103884100/103884100]

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
Note, selecting 'google-chrome-stable' instead of './google-chrome-stable_current_amd64.deb'
google-chrome-stable is already the newest version (119.0.6045.159-1).
0 upgraded, 0 newly installed, 0 to remove and 23 not upgraded.


###Import des librairies

In [11]:
import os
import pandas as pd
import psycopg2
import requests

from base64 import b64decode

from selenium import webdriver
#from selenium.webdriver import Chrome, ChromeOptions
#from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
#from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
#from selenium.webdriver.support import expected_conditions as EC
#from selenium.webdriver.support.ui import WebDriverWait

from time import sleep

###Fonctions utilitaires

In [12]:
def chrome_driver_setup():
    options = webdriver.ChromeOptions()
    #run Selenium in headless mode
    options.add_argument('--headless')
    options.add_argument('--no-sandbox')
    #overcome limited resource problems
    options.add_argument('--disable-dev-shm-usage')
    options.add_argument("lang=en")
    #open Browser in maximized mode
    options.add_argument("start-maximized")
    #disable infobars
    options.add_argument("disable-infobars")
    #disable extension
    options.add_argument("--disable-extensions")
    options.add_argument("--incognito")
    options.add_argument("--disable-blink-features=AutomationControlled")

    driver = webdriver.Chrome(options=options)

    driver.execute_script("Object.defineProperty(navigator, 'webdriver', {get: () => undefined});")

    return driver

In [13]:
def js_wait_for_text_and_click(cssSelector, text):
  return "const interval = setInterval(() => {" \
    "	const element = document.querySelector('" + cssSelector + "');" \
    " if (element && element.innerHTML.trim() === '" + text + "') {" \
		"   clearInterval(interval);" \
		"   element.click();" \
    "	}" \
    "}, 100);"

##Scraping des données

In [14]:
def scrape_country(driver, country):
  print(f"Accessing IEA's page for {country}...")
  driver.get(f"https://www.iea.org/countries/{country}")

  print('   Retrieving category button...')
  #WebDriverWait(driver, 15).until(lambda driver: driver.find_element(By.CSS_SELECTOR, '.a-dropdown--indicatorCategory > .a-dropdown__options > div > ul > li:nth-child(9) > button').get_attribute("innerHTML").strip() == 'Electricity and heat')
  sleep(3)
  btn_category = driver.find_element(By.CSS_SELECTOR, '.a-dropdown--indicatorCategory > .a-dropdown__options > div > ul > li:nth-child(9) > button')
  if btn_category.get_attribute('innerHTML').strip() != 'Electricity and heat':
    print(f"Error : unable to retrieve category button for {country}")
    return False

  print('   Clicking on category button...')
  driver.execute_script(js_wait_for_text_and_click('.a-dropdown--indicatorCategory > .a-dropdown__options > div > ul > li:nth-child(9) > button', 'Electricity and heat'))
  sleep(15)

  print('   Retrieving indicator button...')
  btn_indicator = driver.find_element(By.CSS_SELECTOR, '.a-dropdown--selectedIndicator > .a-dropdown__options > div > ul > li:nth-child(6) > button')
  if btn_indicator.get_attribute('innerHTML').strip() != 'Electricity generation by source':
    print(f"Error : unable to retrieve indicator button for {country}")
    return False

  print('   Clicking on indicator button...')
  driver.execute_script(js_wait_for_text_and_click('.a-dropdown--selectedIndicator > .a-dropdown__options > div > ul > li:nth-child(6) > button', 'Electricity generation by source'))
  sleep(6)

  print('   Retrieving chart title...')
  chart_title = driver.find_element(By.CSS_SELECTOR, 'h3.f-ui-2-bold')
  if chart_title.get_attribute('innerHTML').strip()[:32] != 'Electricity generation by source':
    print(f"Error : unable to retrieve chart title for {country}")
    return False

  print('   Waiting for the chart to be generated...')
  sleep(6)

  print("   Retrieveing chart's data")
  data = driver.execute_script("""const chart = Highcharts.charts[Highcharts.charts.length-1];
    let result = [];
    for(let i = 0; i < chart.series.length; i++) {
      const row = {
            name: chart.series[i].name,
            dataX: chart.series[i].processedXData,
            dataY: chart.series[i].processedYData
        };
      result.push(row);
    }
    return result;""")

  return data

In [15]:
driver = chrome_driver_setup()

countries = {'austria':'AT', 'belgium':'BE'}#, 'czech-republic':'CZ', 'germany':'DE', 'spain':'ES', 'france':'FR', 'hungary':'HU', 'italy':'IT', 'the-netherlands':'NL', 'poland':'PL', 'sweden':'SE'}
energy_sources = []

for country, country_code in countries.items():
  data = scrape_country(driver, country)
  if data:
    energy_sources.append({country_code: data})

driver.quit()

Accessing IEA's page for austria...
   Retrieving category button...
   Clicking on category button...
   Retrieving indicator button...
   Clicking on indicator button...
   Retrieving chart title...
   Waiting for the chart to be generated...
   Retrieveing chart's data
Accessing IEA's page for belgium...
   Retrieving category button...
   Clicking on category button...
   Retrieving indicator button...
   Clicking on indicator button...
   Retrieving chart title...
   Waiting for the chart to be generated...
   Retrieveing chart's data
Accessing IEA's page for czech-republic...
   Retrieving category button...
   Clicking on category button...
   Retrieving indicator button...
   Clicking on indicator button...
   Retrieving chart title...
   Waiting for the chart to be generated...
   Retrieveing chart's data
Accessing IEA's page for germany...
   Retrieving category button...
   Clicking on category button...


KeyboardInterrupt: ignored

###Augmentation des données
Ajout d'une colonne "CO2e" avec les émissions de GES associées à la source d'énergie primaire

In [16]:
# Création du dictionnaire associant source d'énergie et émissions de GES
co2e_dict = {
    'Natural gas':443,
    'Coal':1058,
    'Biofuels':32,
    'Hydro':10,
    'Oil':778,
    'Nuclear':6,
    'Wind':9.864583,
    'Solar PV':43.9,
    'Geothermal':38,
    'Waste':494,
    'Solar thermal':22,
    'Tide':22,
    'Other sources':411.74
}

###Création du dataframe résultant du scraping

In [17]:
df_energy_sources = pd.DataFrame(columns=['Country', 'Energy', 'PowerGlobal', 'CO2e', 'Date'])
for data_country in energy_sources:
  for country, data1 in data_country.items():
    for data2 in data1:
      for i in range(len(data2['dataX'])):
        row = {'Country':[country], 'Energy':[data2['name']], 'PowerGlobal':[data2['dataY'][i]], 'CO2e': [co2e_dict[data2['name']]], 'Date': [str(data2['dataX'][i])+'0101']}
        df_energy_sources = pd.concat([df_energy_sources, pd.DataFrame(row)], ignore_index=True)

display(df_energy_sources)

,Country,Energy,PowerGlobal,CO2e,Date
0,AT,Coal,7006,1058,19900101
1,AT,Coal,7786,1058,19910101
2,AT,Coal,5091,1058,19920101
3,AT,Coal,3740,1058,19930101
4,AT,Coal,3908,1058,19940101
...,...,...,...,...,...
841,CZ,Solar PV,2193,43.9,20170101
842,CZ,Solar PV,2359,43.9,20180101
843,CZ,Solar PV,2312,43.9,20190101
844,CZ,Solar PV,2287,43.9,20200101


###Augmentation des données
Ajout d'une colonne "PowerRelative" représentant la part relative de la source d'énergie primaire dans le mix de production électrique.

In [18]:
energy_sources_power_totals = df_energy_sources.groupby(['Country','Date'])['PowerGlobal'].sum()
df_energy_sources_final = df_energy_sources.merge(energy_sources_power_totals, left_on=['Country', 'Date'], right_on=['Country', 'Date'], how='left')
df_energy_sources_final.PowerGlobal_y = df_energy_sources_final.PowerGlobal_x/df_energy_sources_final.PowerGlobal_y*100
df_energy_sources_final = df_energy_sources_final.rename(columns={'PowerGlobal_x':'PowerGlobal','PowerGlobal_y':'PowerRelative'})
df_energy_sources_final = df_energy_sources_final[['Country', 'Energy', 'PowerGlobal', 'PowerRelative', 'CO2e', 'Date']]
display(df_energy_sources_final)

,Country,Energy,PowerGlobal,PowerRelative,CO2e,Date
0,AT,Coal,7006,13.930091,1058,19900101
1,AT,Coal,7786,15.123439,1058,19910101
2,AT,Coal,5091,9.945302,1058,19920101
3,AT,Coal,3740,7.134545,1058,19930101
4,AT,Coal,3908,7.355266,1058,19940101
...,...,...,...,...,...,...
841,CZ,Solar PV,2193,2.519271,43.9,20170101
842,CZ,Solar PV,2359,2.679738,43.9,20180101
843,CZ,Solar PV,2312,2.656494,43.9,20190101
844,CZ,Solar PV,2287,2.805619,43.9,20200101


###Sauvegarde des données en CSV

In [ ]:
df_energy_sources_final.to_csv('./energy_sources.csv', index=False)

##Import dans la base de données

###Récupération des données depuis le CSV

In [ ]:
df_energy_sources_final = pd.read_csv('./energy_sources.csv')

###Remplacement des valeurs nulles par des 0

In [ ]:
df_energy_sources_final.fillna(0, inplace=True)
display(df_energy_sources_final)

###Récupération de l'adresse IP du notebook
Permet de whitelister l'adresse pour autoriser l'accès à la base de données.

In [ ]:
try:
  response = requests.get('https://api.ipify.org/?format=json', timeout=10)
  print("Adresse IP publique :", response.json()['ip'])
except requests.RequestException:
  print("Impossible de récupérer l'adresse IP publique")

###Définition des constantes

In [ ]:
PG_HOST = 'rncp-ol.postgres.database.azure.com'
PG_PORT = 5432
PG_DATABASE = 'rncp_prod'
PG_USERNAME = 'm2i'
PG_PASSWORD = 'Uk5DUF9vbDIwMjM='
PG_CONNECT_TIMEOUT = 10
PG_BATCH_MAX_SIZE = 100
PG_TABLE_ENERGY_SOURCES = 'co2_energy_sources'

###Connexion à la base de données

In [ ]:
pg_conn = psycopg2.connect(
    host=PG_HOST,
    port=PG_PORT,
    database=PG_DATABASE,
    user=PG_USERNAME,
    password=b64decode(PG_PASSWORD).decode('utf-8'),
    connect_timeout=PG_CONNECT_TIMEOUT
)
print("Connexion à la base de données PostgreSQL établie avec succès")

pg_conn.autocommit = True

pg_cursor = pg_conn.cursor()

###Création de la table si elle n'existe pas

In [ ]:
pg_cursor.execute(f"""CREATE TABLE IF NOT EXISTS {PG_TABLE_ENERGY_SOURCES}
  (
    UID INT NOT NULL,
    Country TEXT NOT NULL,
    Energy TEXT NOT NULL,
    PowerGlobal INT NOT NULL,
    PowerRelative FLOAT NOT NULL,
    CO2e FLOAT NOT NULL,
    Date TEXT NOT NULL,
    CONSTRAINT {PG_TABLE_ENERGY_SOURCES}_pk PRIMARY KEY(UID)
  );""")

###Insertion des données dans la base de données
Insertion par lots de PG_BATCH_MAX_SIZE.

In [ ]:
queries = []
row_index = 1
for index, row in df_energy_sources_final.iterrows():
  queries.append(f"'{row['Country']}', '{row['Energy']}', {row['PowerGlobal']}, {row['PowerRelative']}, {row['CO2e']}, '{row['Date']}'")
  if row_index % PG_BATCH_MAX_SIZE == 0:
    print(f"Inserting batch #{(row_index // PG_BATCH_MAX_SIZE)} into DB...")
    query = f"INSERT INTO {PG_TABLE_ENERGY_SOURCES} (Country, Energy, PowerGlobal, PowerRelative, CO2e, Date) VALUES " + ", ".join([f"({q})" for q in queries])
    pg_cursor.execute(query)
    queries = []
  row_index += 1

if queries != []:
  print(f"Inserting batch #{(row_index // PG_BATCH_MAX_SIZE)} into DB...")
  query = f"INSERT INTO {PG_TABLE_ENERGY_SOURCES} (Country, Energy, PowerGlobal, PowerRelative, CO2e, Date) VALUES " + ", ".join([f"({q})" for q in queries])
  pg_cursor.execute(query)

print('Import into DB completed')

###Clôture de la connexion à la base de données

In [ ]:
pg_cursor.close()
pg_conn.close()